In [5]:
import pandas as pd
from collections import Counter
from rapidfuzz import fuzz
import re
from tqdm import tqdm  # make sure tqdm is installed

# === 1. Load CSV ===
csv_file = "20250903_Extrait_Constatations_F2.csv"
df = pd.read_csv(csv_file, delimiter=';', dtype=str)
df.columns = df.columns.str.strip()
desc_col = "Description"
comment_col = "Commentaire"
df = df.dropna(subset=[desc_col])

# === 2. Preprocess text ===
def clean_words(text):
    if pd.isna(text):
        return []
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    return text.split()

def generate_clusters(words, min_len=1, max_len=2):
    clusters = []
    for n in range(min_len, min(max_len+1, len(words)+1)):
        for i in range(len(words)-n+1):
            clusters.append(' '.join(words[i:i+n]))
    return clusters

# === 3. Extract all clusters from Description ===
desc_clusters_list = []
for desc in df[desc_col]:
    words = clean_words(desc)
    clusters = generate_clusters(words)
    desc_clusters_list.extend(clusters)

# === 4. Cluster similar Description clusters with RapidFuzz ===
desc_groups = {}
threshold = 85  # similarity %

print("Clustering Description clusters with RapidFuzz...")
for cluster in tqdm(desc_clusters_list, desc="Description Clustering"):
    found = False
    for key in desc_groups.keys():
        if fuzz.ratio(cluster, key) >= threshold:
            desc_groups[key].append(cluster)
            found = True
            break
    if not found:
        desc_groups[cluster] = [cluster]

# === 5. Aggregate counts and associated Commentaire clusters ===
results = []
top_n_comments = 10  # keep only top N associated comment clusters
checkpoint_every = 10  # save partial results every 50 groups

print("Finding associated Commentaire clusters...")
for i, (group_key, variants) in enumerate(tqdm(desc_groups.items(), desc="Processing Description Groups")):
    # Rows where any variant appears
    mask = df[desc_col].apply(lambda x: any(v.lower() in x.lower() for v in variants))
    commentaires = df.loc[mask, comment_col].dropna().tolist()

    # Extract clusters from commentaires
    comment_clusters_list = []
    for com in commentaires:
        words = clean_words(com)
        clusters = generate_clusters(words)
        comment_clusters_list.extend(clusters)

    # Cluster similar Commentaire clusters with RapidFuzz
    comment_groups = {}
    for c in comment_clusters_list:
        found = False
        for k in comment_groups.keys():
            if fuzz.ratio(c, k) >= threshold:
                comment_groups[k].append(c)
                found = True
                break
        if not found:
            comment_groups[c] = [c]

    # Keep only top N associated comment clusters
    top_comments = sorted(comment_groups.items(), key=lambda x: len(x[1]), reverse=True)[:top_n_comments]
    associated_comments = '|'.join([f"{k}({len(v)})" for k, v in top_comments])

    # Count occurrences in Description
    count = sum(df[desc_col].str.contains('|'.join([re.escape(v) for v in variants]), case=False, na=False))

    results.append({
        "Description_Cluster": group_key,
        "Count": count,
        "Associated_Comment_Clusters": associated_comments
    })

    # === Checkpoint save every N iterations ===
    if (i + 1) % checkpoint_every == 0:
        temp_df = pd.DataFrame(results)
        temp_df.to_csv("checkpoint_partial_results.csv", sep=';', index=False)
        print(f"Checkpoint saved after {i+1} Description clusters.")

# === 6. Convert to DataFrame and save final CSV ===
summary_df = pd.DataFrame(results).sort_values(by="Count", ascending=False)
summary_df.to_csv("description_comment_clusters_rapidfuzz_top.csv", index=False, sep=';')
display(summary_df)
print("Saved final results as 'description_comment_clusters_rapidfuzz_top.csv'")


Clustering Description clusters with RapidFuzz...


Description Clustering: 100%|██████████████████████████████████████████████████| 127304/127304 [46:09<00:00, 45.96it/s]


Finding associated Commentaire clusters...


Processing Description Groups:   0%|                                             | 10/14883 [01:53<31:39:16,  7.66s/it]

Checkpoint saved after 10 Description clusters.


...

Checkpoint saved after 14870 Description clusters.


Processing Description Groups: 100%|█████████████████████████████████████████████| 14883/14883 [39:58<00:00,  6.20it/s]

Checkpoint saved after 14880 Description clusters.


,Description_Cluster,Count,Associated_Comment_Clusters
91,e,7407,de(400)|d(147)|fissures(139)|sup(132)|non(131)...
438,r,6978,de(390)|fissures(139)|d(136)|non(133)|sup(131)...
1584,n,6887,de(388)|d(141)|fissures(127)|sup(123)|non(122)...
5,s,6626,de(373)|d(135)|fissures(134)|non(133)|sup(130)...
3,t,6608,de(368)|d(144)|fissures(133)|non(129)|sup(129)...
...,...,...,...
14838,ngrisse netzrissartig,0,
14837,schraubenk pfe,0,
14836,sehr grossem,0,
14835,sichtbaren bewehrungseisen,0,


Saved final results as 'description_comment_clusters_rapidfuzz_top.csv'
